<a href="https://colab.research.google.com/github/sharadnatarajSJSU/CMPE_255_MovieReccomend/blob/main/Movie_recommendation_SOTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [ ]:
!pip install awscli

In [ ]:
import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/MyDrive/config/config.ini

In [ ]:
path = "/content/drive/My Drive/config/config.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

/content/drive/My Drive/config/config.ini


In [ ]:
!aws s3 ls s3://255movie --recursive --human-readable --summarize

2021-12-11 00:20:56  206.2 MiB movies.csv

Total Objects: 1
   Total Size: 206.2 MiB


In [ ]:
import boto3.session

import json
import numpy as np
import pandas as pd
import time

In [ ]:
personalize = boto3.client('personalize',aws_region_name='us-west-2')


In [ ]:
personalize_runtime = boto3.client('personalize-runtime',region_name='us-west-2')

In [ ]:
bucket = "255movie"
filename = "movies.csv"

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip -o ml-20m.zip
data = pd.read_csv('./ml-20m/ratings.csv', sep=',', dtype={'userid': "int64", 'movieid': "int64", 'rating': "float64", 'timestamp': "int64"})
pd.set_option('display.max_rows', 25)
data.rename(columns = {'userId':'USER_ID','movieId':'ITEM_ID','rating':'RATING','timestamp':'TIMESTAMP'}, inplace = True)
data

--2021-12-11 00:40:39--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘ml-20m.zip’ not modified on server. Omitting download.

Archive:  ml-20m.zip
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
20000258,138493,68954,4.5,1258126920
20000259,138493,69526,4.5,1259865108
20000260,138493,69644,3.0,1260209457
20000261,138493,70286,5.0,1258126944


In [ ]:
data = data[data['RATING'] > 3.6] 
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']]
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}



In [ ]:
create_schema_response = personalize.create_schema(
    name = "DEMO-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-west-2:332571597467:schema/DEMO-schema",
  "ResponseMetadata": {
    "RequestId": "5b90af55-0c76-4f88-bff8-aabd946d86ee",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 11 Dec 2021 00:50:24 GMT",
      "x-amzn-requestid": "5b90af55-0c76-4f88-bff8-aabd946d86ee",
      "content-length": "77",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-dataset-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-west-2:332571597467:dataset-group/DEMO-dataset-group",
  "ResponseMetadata": {
    "RequestId": "a1ef6539-fead-4910-9fec-7ec05b6a6f6a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 11 Dec 2021 00:50:56 GMT",
      "x-amzn-requestid": "a1ef6539-fead-4910-9fec-7ec05b6a6f6a",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: ACTIVE


In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "DEMO-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-west-2:332571597467:dataset/DEMO-dataset-group/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "b3b3e5a9-6e40-40a7-8c23-17ff14b4e52d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 11 Dec 2021 00:51:15 GMT",
      "x-amzn-requestid": "b3b3e5a9-6e40-40a7-8c23-17ff14b4e52d",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
s3 = boto3.client("s3",region_name='us-west-2')

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Sat, 11 Dec 2021 00:51:55 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': '3xk4yWKLJjaOX/I/iQronRJRL56LQoZTxomJuzrlBD+gfEko4hbNUWkYlJIK29lhpw1zrn0ipBY=',
   'x-amz-request-id': 'C2ZRNPJZGWCGNRWF'},
  'HTTPStatusCode': 204,
  'HostId': '3xk4yWKLJjaOX/I/iQronRJRL56LQoZTxomJuzrlBD+gfEko4hbNUWkYlJIK29lhpw1zrn0ipBY=',
  'RequestId': 'C2ZRNPJZGWCGNRWF',
  'RetryAttempts': 1}}

In [ ]:
iam = boto3.client("iam",region_name='us-west-2')

role_name = "PersonalizeRole"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess'
)

time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::332571597467:role/PersonalizeRole


In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-dataset-import-job",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = 'arn:aws:iam::332571597467:role/PersonalizeRole'
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-west-2:332571597467:dataset-import-job/DEMO-dataset-import-job",
  "ResponseMetadata": {
    "RequestId": "0e41e060-bdc8-4530-a552-d2f808290e23",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 11 Dec 2021 01:03:11 GMT",
      "x-amzn-requestid": "0e41e060-bdc8-4530-a552-d2f808290e23",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [ ]:
list_recipes_response = personalize.list_recipes()
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn" # aws-hrnn selected for demo purposes
list_recipes_response

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '3735',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 11 Dec 2021 00:54:18 GMT',
   'x-amzn-requestid': '086aa7e7-f8b4-43e0-8e41-33a6e53c6f6a'},
  'HTTPStatusCode': 200,
  'RequestId': '086aa7e7-f8b4-43e0-8e41-33a6e53c6f6a',
  'RetryAttempts': 0},
 'nextToken': 'AYADeE5C54pXhhVI2OGhKLnq0bwAXwABABVhd3MtY3J5cHRvLXB1YmxpYy1rZXkAREFyc2hESm5Dc1kvdERjZUxDYVg4UzZseExCWWhCL0tLVWZFakJGclhhZ3ExN1NCdW1SWjdkbEJobzdsZ1EvWDVyUT09AAEAB2F3cy1rbXMAS2Fybjphd3M6a21zOnVzLXdlc3QtMjo2MDI0NDY3NTc1Nzk6a2V5L2YzZDcwOGM3LWYxMTctNGIyYi1hMTA5LWMxMzg2YzNkOTlkYgC4AQIBAHg0rvLpwtlSGYfQK/DAtr8RXDNOB93tMwdSQOZ5E4BI2AEo63grDJFrDJ4sEpEQsGVTAAAAfjB8BgkqhkiG9w0BBwagbzBtAgEAMGgGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMAT6cYI7EXVuPnNFcAgEQgDtJgDIBulhabYZePudVf2lHWKXF/o7p3wbyWlCixTDvjFhZU4riAWh87bd2Byekm39c1iQ1BtERK2dUbwIAAAAADAAAEAAAAAAAAAAAAAAAAABxsLYw3dq3ETPYp2Kz+1IB/////wAAAAEAAAAAAAAAAAAAAAEAAABz7slq+gYmPGJ4XP9GTMoewap9GD

In [ ]:
create_solution_response = personalize.create_solution(
    name = "DEMO-solution",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-west-2:332571597467:solution/DEMO-solution",
  "ResponseMetadata": {
    "RequestId": "87d9f143-153c-4941-a68c-a92cf4a83023",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 11 Dec 2021 01:08:53 GMT",
      "x-amzn-requestid": "87d9f143-153c-4941-a68c-a92cf4a83023",
      "content-length": "83",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-west-2:332571597467:solution/DEMO-solution/a2f9aba5",
  "ResponseMetadata": {
    "RequestId": "a30f192a-771c-410a-95d3-46a6ab1c0f2d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 11 Dec 2021 01:51:40 GMT",
      "x-amzn-requestid": "a30f192a-771c-410a-95d3-46a6ab1c0f2d",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
  describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = solution_version_arn
    )
  status = describe_solution_version_response["solutionVersion"]["status"]
  print("SolutionVersion: {}".format(status))

  if status == "ACTIVE" or status == "CREATE FAILED":
    break
        
  time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: ACTIVE


In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-west-2:332571597467:solution/DEMO-solution/a2f9aba5",
  "metrics": {
    "coverage": 0.2244,
    "mean_reciprocal_rank_at_25": 0.1915,
    "normalized_discounted_cumulative_gain_at_10": 0.2045,
    "normalized_discounted_cumulative_gain_at_25": 0.2752,
    "normalized_discounted_cumulative_gain_at_5": 0.1598,
    "precision_at_10": 0.0577,
    "precision_at_25": 0.0429,
    "precision_at_5": 0.0693
  },
  "ResponseMetadata": {
    "RequestId": "5e0b60d9-af9c-4e5b-a827-2470539779b4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 11 Dec 2021 02:49:21 GMT",
      "x-amzn-requestid": "5e0b60d9-af9c-4e5b-a827-2470539779b4",
      "content-length": "397",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-campaign",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-west-2:332571597467:campaign/DEMO-campaign",
  "ResponseMetadata": {
    "RequestId": "d272b255-b510-49fa-8052-465ef63bd9f4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sat, 11 Dec 2021 02:49:31 GMT",
      "x-amzn-requestid": "d272b255-b510-49fa-8052-465ef63bd9f4",
      "content-length": "83",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


In [ ]:
items = pd.read_csv('./ml-20m/movies.csv', sep=',', usecols=[0,1], header=0)
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

items

USER: 105572
ITEM: Forrest Gump (1994)


,ITEM_ID,TITLE
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
27273,131254,Kein Bund für's Leben (2007)
27274,131256,"Feuer, Eis & Dosenbier (2002)"
27275,131258,The Pirates (2014)
27276,131260,Rentun Ruusu (2001)


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
    itemId = str(item_id)
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

Recommendations: [
  "Searching for Bobby Fischer (1993)",
  "In the Name of the Father (1993)",
  "When a Man Loves a Woman (1994)",
  "Rock, The (1996)",
  "Just Cause (1995)",
  "Murder in the First (1995)",
  "Man Without a Face, The (1993)",
  "Boys on the Side (1995)",
  "Nobody's Fool (1994)",
  "Snow White and the Seven Dwarfs (1937)",
  "What's Love Got to Do with It? (1993)",
  "Paper, The (1994)",
  "Postman, The (Postino, Il) (1994)",
  "Circle of Friends (1995)",
  "Hudsucker Proxy, The (1994)",
  "Something to Talk About (1995)",
  "Much Ado About Nothing (1993)",
  "Walk in the Clouds, A (1995)",
  "To Die For (1995)",
  "River Wild, The (1994)",
  "Immortal Beloved (1994)",
  "Corrina, Corrina (1994)",
  "Remains of the Day, The (1993)",
  "Independence Day (a.k.a. ID4) (1996)",
  "Truth About Cats & Dogs, The (1996)"
]
